In [0]:
!pip install tensorflow==2.0.0

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/KrylovS/sign-language-translation/master/content/sign_mnist_train.csv \
    -O /content/sign_mnist_train.csv

!wget --no-check-certificate \
    https://raw.githubusercontent.com/KrylovS/sign-language-translation/master/content/sign_mnist_test.csv \
    -O /content/sign_mnist_test.csv

In [0]:
train_df = pd.read_csv('/content/sign_mnist_train.csv')
test_df = pd.read_csv('/content/sign_mnist_test.csv')

In [0]:
train_df.head()

In [0]:
labels = train_df['label'].to_numpy()

In [0]:
labels_list = np.unique(labels).tolist()

In [0]:
labels_to_chars = {i : chr(ord('a') + i) for i in labels_list}

In [0]:
train = train_df.drop('label', axis=1).to_numpy()

In [0]:
plt.imshow(train[0].reshape(28,28))

In [0]:
def plot_images(original_img, labels, images_count, labels_to_chars):
  plt.figure(figsize=(20, 5))
  for i in range(images_count): 
      ax = plt.subplot(1, images_count, i + 1)
      ax.title.set_text(labels_to_chars[labels[i]])
      plt.imshow(original_img[i].reshape(28, 28))
      plt.gray()
      plt.axis('off')
  plt.show()
  

In [0]:
plot_images(train, labels, 5, labels_to_chars)

In [0]:
from sklearn.preprocessing import LabelBinarizer
label_binrizer = LabelBinarizer()
labels_enc = label_binrizer.fit_transform(labels)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train, labels_enc, 
                                                    test_size = 0.3, random_state = 13)

In [0]:
y_train.shape

In [0]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1) / 255

In [0]:
x_train.shape

In [0]:
#!pip install tensorflow==1.15.0
!pip install tensorflow==2.0.0

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Reshape
from tensorflow.keras import Model

In [0]:
model = tf.keras.models.load_model("/content/simplemodel2.h5")

In [0]:
model1 = tf.keras.models.Sequential([
                                    #Reshape((28, 28, 1), input_shape=(784,)),
                                    Conv2D(32, (3,3), activation = 'relu', input_shape=(28,28,1)),
                                    MaxPooling2D(2, 2),

                                    Conv2D(64, (3,3), activation = 'relu'),
                                    MaxPooling2D(2, 2),

                                    Conv2D(128, (3,3), activation = 'relu'),
                                    MaxPooling2D(2, 2),

                                    Flatten(),
                                    Dense(128, activation = 'relu'),
                                    Dropout(0.2),
                                    Dense(24, activation = 'softmax')
])

In [0]:
model2 = tf.keras.models.Sequential([
                                    #Reshape((28, 28, 1), input_shape=(784,)),
                                    Conv2D(32, (3,3), activation = 'relu', input_shape=(28,28,1), padding = "same"),
                                    MaxPooling2D(2, 2),

                                    Conv2D(64, (3,3), activation = 'relu', padding = "same"),
                                    Dropout(0.2),
                                    MaxPooling2D(2, 2),

                                    Conv2D(128, (3,3), activation = 'relu', padding = "same"),
                                    Dropout(0.2), 
                                    MaxPooling2D(2, 2),
                                     
                                    Conv2D(128, (3,3), activation = 'relu', padding = "same"),
                                    Dropout(0.2), 
                                    MaxPooling2D(2, 2),

                                    Flatten(),
                                    Dense(128, activation = 'relu'),
                                    Dropout(0.2),
                                    Dense(24, activation = 'softmax')
])

In [0]:
#model = model1
#model1.summary()
model = model2
model.summary()

In [0]:
class userCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > 0.999):
      print("\n Reached accuracy. Stop trainning!")
      self.model.stop_training = True

In [0]:
callback = userCallback()
batch_size = 256
epochs = 20

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs = epochs, validation_data = (x_val, y_val),
                    callbacks = [callback], batch_size = batch_size)

In [0]:
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(len(acc_train))

plt.plot(epochs, acc_train)
plt.plot(epochs, acc_val)
plt.legend(['train','val'])
plt.show()

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [0]:
labels_test = test_df['label'].to_numpy()
x_test = test_df.drop('label', axis = 1).to_numpy()
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255

In [0]:
label_binrizer = LabelBinarizer()
labels_test_enc = label_binrizer.fit_transform(labels_test)

In [0]:
pred = model.predict(x_test)

In [0]:
accuracy_score(labels_test_enc, pred.round())

In [0]:
f1_score(labels_test_enc, pred.round(),average = 'weighted')

In [0]:
model.save("simplemodel3.h5")

In [0]:
#model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (32, 32, 3))


In [0]:
x_train.shape

In [0]:
x_train.reshape(x_train.shape[0],28,28).shape

In [0]:
def prepare_for_model(x):
  x = x.reshape(x.shape[0],28,28) / 255
  x = np.pad(x, ((0, 0),(2, 2),(2, 2)), constant_values=0)
  x = np.repeat(x[:, :, :, np.newaxis], 3, axis=3)
  return x

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train, labels_enc, 
                                                    test_size = 0.3, random_state = 13)

x_train = prepare_for_model(x_train)
x_val = prepare_for_model(x_val)

In [0]:
x_train.shape

In [0]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1) / 255

In [0]:
plt.imshow(b[0,:,:,0])

In [0]:
import tensorflow.keras.applications as applications
model = applications.resnet.ResNet50(weights = "imagenet", include_top=False, input_shape = (32, 32, 3))

In [0]:
model.summary()

In [0]:
for layer in model.layers[:-4]:
    layer.trainable = False

In [0]:
for layer in model.layers:
    print(layer, layer.trainable)

In [0]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(24, activation="softmax")(x)
model = Model(inputs = model.inputs, outputs = predictions)
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
callback = userCallback()
batch_size = 256
epochs = 20

In [0]:

history = model.fit(x_train, y_train, epochs = epochs, validation_data = (x_val, y_val),
                    callbacks = [callback], batch_size = batch_size)

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [0]:
labels_test = test_df['label'].to_numpy()
x_test = test_df.drop('label', axis = 1).to_numpy()
x_test = prepare_for_model(x_test)

In [0]:
label_binrizer = LabelBinarizer()
labels_test_enc = label_binrizer.fit_transform(labels_test)

In [0]:
pred = model.predict(x_test)